# Importing Required Libraries

In [2]:
!python -m pip install --upgrade pip

  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


In [14]:
!pip install IPy

  Running setup.py bdist_wheel for IPy: started
  Running setup.py bdist_wheel for IPy: finished with status 'done'
  Stored in directory: C:\Users\Hamza\AppData\Local\pip\Cache\wheels\51\9a\f1\32eeda4d895db4153e86b59a4ffd4582d344d96c66b5459788
Successfully built IPy


In [3]:
!pip install https://files.pythonhosted.org/packages/dc/f2/9e1a2953d4d824e183ac033e3d223055e40e695fa6db2cb3e94a864eaa84/python-nmap-0.6.1.tar.gz

  Running setup.py bdist_wheel for python-nmap: started
  Running setup.py bdist_wheel for python-nmap: finished with status 'done'
  Stored in directory: C:\Users\Hamza\AppData\Local\pip\Cache\wheels\cb\de\b1\41e388f95afba300c0840ef312601d75c91100681a08c56284
Successfully built python-nmap


In [1]:
from socket import *
import struct
import sys
import re
import platform
from nmap import *
import time
import errno
import os
import argparse
import subprocess
from IPy import IP

# Socket Binding 

In [2]:
HOST = gethostbyname(gethostname())

# create a raw socket and bind it to the public interface
s = socket(AF_INET, SOCK_RAW, IPPROTO_IP)
s.bind((HOST, 0))
s.setsockopt(IPPROTO_IP, IP_HDRINCL, 1)
s.ioctl(SIO_RCVALL, RCVALL_ON)

OSError: [WinError 10045] The attempted operation is not supported for the type of object referenced

# Receiving a Datagram

In [3]:
def receiveData(s):
    data = ''
    try:
        data = s.recvfrom(65565)
    except timeout:
        data = ''
    except:
        print ("An error happened: ")
        sys.exc_info()
    return data[0]

# Analyzing IP header


# Determining Type of Service 

In [4]:
def getTOS(data):
    precedence = {0: "Routine", 1: "Priority", 2: "Immediate", 3: "Flash", 4: "Flash override", 5: "CRITIC/ECP",
                  6: "Internetwork control", 7: "Network control"}
    delay = {0: "Normal delay", 1: "Low delay"}
    throughput = {0: "Normal throughput", 1: "High throughput"}
    reliability = {0: "Normal reliability", 1: "High reliability"}
    cost = {0: "Normal monetary cost", 1: "Minimize monetary cost"}

    #   get the 3rd bit and shift right
    D = data & 0x10
    D >>= 4
    #   get the 4th bit and shift right
    T = data & 0x8
    T >>= 3
    #   get the 5th bit and shift right
    R = data & 0x4
    R >>= 2
    #   get the 6th bit and shift right
    M = data & 0x2
    M >>= 1
    #   the 7th bit is empty and shouldn't be analyzed

    tabs = '\n\t\t\t'
    TOS = precedence[data >> 5] + tabs + delay[D] + tabs + throughput[T] + tabs + \
          reliability[R] + tabs + cost[M]
    return TOS

# Determining Flags

In [5]:
def getFlags(data):
    flagR = {0: "0 - Reserved bit"}
    flagDF = {0: "0 - Fragment if necessary", 1: "1 - Do not fragment"}
    flagMF = {0: "0 - Last fragment", 1: "1 - More fragments"}

    #   get the 1st bit and shift right
    R = data & 0x8000
    R >>= 15
    #   get the 2nd bit and shift right
    DF = data & 0x4000
    DF >>= 14
    #   get the 3rd bit and shift right
    MF = data & 0x2000
    MF >>= 13

    tabs = '\n\t\t\t'
    flags = flagR[R] + tabs + flagDF[DF] + tabs + flagMF[MF]
    return flags


In [6]:
data = receiveData(s)

# get the IP header (the first 20 bytes) and unpack them
# B - unsigned char (1)
# H - unsigned short (2)
# s - string
unpackedData = struct.unpack('!BBHHHBBH4s4s', data[:20])

version_IHL = unpackedData[0]
version = version_IHL >> 4  # version of the IP
IHL = version_IHL & 0xF  # internet header length
offset = IHL * 4
TOS = unpackedData[1]  # type of service
totalLength = unpackedData[2]
ID = unpackedData[3]  # identification
flags = unpackedData[4]
fragmentOffset = unpackedData[4] & 0x1FFF
TTL = unpackedData[5]  # time to live
protocol2 = unpackedData[6]
checksum = unpackedData[7]
sourceAddress = inet_ntoa(unpackedData[8])
destinationAddress = inet_ntoa(unpackedData[9])

print ("An IP packet with the size %i was captured." % (unpackedData[2]))
print ("Raw data: " ,    data)
print ("\nParsed data")
print ("Version:\t\t" + str(version))
print ("Header Length:\t\t" + str(IHL * 4) + " bytes")
print ("Type of Service:\t" + getTOS(TOS))
print ("Length:\t\t\t" + str(totalLength))
print ("ID:\t\t\t" + str(hex(ID)) + " (" + str(ID) + ")")
print ("Flags:\t\t\t" + getFlags(flags))
print ("Fragment offset:\t" + str(fragmentOffset))
print ("TTL:\t\t\t" + str(TTL))
print ("Protocol:\t\t" + str(protocol2))
print ("Checksum:\t\t" + str(checksum))
print ("Source:\t\t\t" + sourceAddress)
print ("Destination:\t\t" + destinationAddress)
#print "Payload:\n" + data[20:])
print ("Header-data offset:\t" + str(offset))

An IP packet with the size 52 was captured.
Raw data:  b'E\x00\x004m\x85@\x00\x80\x06\xa4FoX\xa7=j3h/\x14b\x12\xa5\xde\xbb\x17\xa9\xb5\xd2\xf1\xc2\x80\x10\x01\x03{\x99\x00\x00\x01\x01\x05\n\xb5\xd2\xf1\xbe\xb5\xd2\xf1\xc2'

Parsed data
Version:		4
Header Length:		20 bytes
Type of Service:	Routine
			Normal delay
			Normal throughput
			Normal reliability
			Normal monetary cost
Length:			52
ID:			0x6d85 (28037)
Flags:			0 - Reserved bit
			1 - Do not fragment
			0 - Last fragment
Fragment offset:	0
TTL:			128
Protocol:		6
Checksum:		42054
Source:			111.88.167.61
Destination:		106.51.104.47
Header-data offset:	20


# Analyzing TCP/UDP Header

In [7]:
#for tcp/udp packet
if protocol2 == 6:
    unpackedtcpData = struct.unpack('!HHLLHH', data[offset+1:offset+17])
    source_port = unpackedtcpData[0]
    destination_port = unpackedtcpData[1]
    sequence_number = unpackedtcpData[2]
    acnowledgment_number = unpackedtcpData[3]
    offset_reserved_flag = unpackedtcpData[4]
    window_size = unpackedtcpData[5]
    print ("TCP protocol is used in transport layer")
    print ("source port no:\t\t\t" + str(source_port))
    print ("destination port no:\t\t" + str(destination_port))
    print ("sequence number:\t\t" + str(sequence_number))
    print ("acknowledgment number:\t\t" + str(acnowledgment_number))
    print ("window size:\t\t\t" + str(window_size))

if protocol2==17:
    print ("UDP protocol is used in transport layer")

TCP protocol is used in transport layer
source port no:			25106
destination port no:		42462
sequence number:		3138890165
acknowledgment number:		3539059328
window size:			891


# OS Finger-printing

# For a directly reachable IP address

In [ ]:
nm = nmap.PortScanner() 
nm.scan('127.0.0.1', '22-443',arguments="-O")      # scan host 127.0.0.1, ports from 22 to 443
nm.command_line()                   # get command line used for the scan : nmap -oX - -p 22-443 127.0.0.1
nm.scaninfo()                       # get nmap scan informations {'tcp': {'services': '22-443', 'method': 'connect'}}
nm.all_hosts()                      # get all hosts that were scanned
#nm['127.0.0.1'].hostname()          # get one hostname for host 127.0.0.1, usualy the user record
nm['127.0.0.1'].hostnames()         # get list of hostnames for host 127.0.0.1 as a list of dict [{'name':'hostname1', 'type':'PTR'}, {'name':'hostname2', 'type':'user'}]
nm['127.0.0.1'].state()             # get state of host 127.0.0.1 (up|down|unknown|skipped) 
nm['127.0.0.1'].all_protocols()     # get all scanned protocols ['tcp', 'udp'] in (ip|tcp|udp|sctp)
if ('tcp' in nm['127.0.0.1']):
    list(nm['127.0.0.1']['tcp'].keys()) # get all ports for tcp protocol

nm['127.0.0.1'].all_tcp()           # get all ports for tcp protocol (sorted version)
nm['127.0.0.1'].all_udp()           # get all ports for udp protocol (sorted version)
nm['127.0.0.1'].all_ip()            # get all ports for ip protocol (sorted version)
nm['127.0.0.1'].all_sctp()          # get all ports for sctp protocol (sorted version)
if nm['127.0.0.1'].has_tcp(22):     # is there any information for port 22/tcp on host 127.0.0.1
    nm['127.0.0.1']['tcp'][22]          # get infos about port 22 in tcp on host 127.0.0.1
    nm['127.0.0.1'].tcp(22)             # get infos about port 22 in tcp on host 127.0.0.1
    nm['127.0.0.1']['tcp'][22]['state'] # get state of port 22/tcp on host 127.0.0.1 (open


# a more usefull example :
for host in nm.all_hosts():
    print('----------------------------------------------------')
    print('Host : {0} ({1})'.format(host, nm[host].hostname()))
    print('State : {0}'.format(nm[host].state()))

    for proto in nm[host].all_protocols():
        print('----------')
        print('Protocol : {0}'.format(proto))

        lport = list(nm[host][proto].keys())
        lport.sort()
        for port in lport:
            print('port : {0}\tstate : {1}'.format(port, nm[host][proto][port]))


print('----------------------------------------------------')
# print result as CSV
print(nm.csv())



print('----------------------------------------------------')
# If you want to do a pingsweep on network 192.168.1.0/24:
nm.scan(hosts='192.168.0.0/24', arguments='-n -sP -PE -PA21,23,80,3389')
hosts_list = [(x, nm[x]['status']['state']) for x in nm.all_hosts()]
for host, status in hosts_list:
    print('{0}:{1}'.format(host, status))





print('----------------------------------------------------')
# Asynchronous usage of PortScannerAsync


nma = nmap.PortScannerAsync()

def callback_result(host, scan_result):
    print('------------------')
    print(host, scan_result)

nma.scan(hosts='192.168.0.0/30', arguments='-sP', callback=callback_result)

while nma.still_scanning():
    print("Waiting ...")
    nma.wait(2)   # you can do whatever you want but I choose to wait after the end of the scan

if (True):
    print('----------------------------------------------------')
    # Os detection (need root privileges)
    nm.scan("127.0.0.1", arguments="-O")
    if 'osclass' in nm['127.0.0.1']:
        for osclass in nm['127.0.0.1']['osclass']:
            print('OsClass.type : {0}'.format(osclass['type']))
            print('OsClass.vendor : {0}'.format(osclass['vendor']))
            print('OsClass.osfamily : {0}'.format(osclass['osfamily']))
            print('OsClass.osgen : {0}'.format(osclass['osgen']))
            print('OsClass.accuracy : {0}'.format(osclass['accuracy']))
            print('')

    if 'osmatch' in nm['127.0.0.1']:
        for osmatch in nm['127.0.0.1']['osmatch']:
            print('OsMatch.name : {0}'.format(osclass['name']))
            print('OsMatch.accuracy : {0}'.format(osclass['accuracy']))
            print('OsMatch.line : {0}'.format(osclass['line']))
            print('')

    if 'fingerprint' in nm['127.0.0.1']:
        print('Fingerprint : {0}'.format(nm['127.0.0.1']['fingerprint']))


    # Vendor list for MAC address
    nm.scan('192.168.0.0/24', arguments='-O')
    for h in nm.all_hosts():
        if 'mac' in nm[h]['addresses']:
            print(nm[h]['addresses'], nm[h]['vendor'])



print('----------------------------------------------------')
# Read output captured to a file
# Example : nmap -oX - -p 22-443 -sV 127.0.0.1 > nmap_output.xml

print(nm.csv())



print('----------------------------------------------------')
# Progressive scan with generator
nm = nmap.PortScannerYield()
for progressive_result in nm.scan('127.0.0.1/24', '22-25'):
    print(progressive_result)

# For packet's Source IP and Source Port Number

In [ ]:
nm = nmap.PortScanner() 
nm.scan(sourceAddress, str(source_port),arguments="-O")      # scan host 127.0.0.1, ports from 22 to 443
nm.command_line()                   # get command line used for the scan : nmap -oX - -p 22-443 127.0.0.1
nm.scaninfo()                       # get nmap scan informations {'tcp': {'services': '22-443', 'method': 'connect'}}
nm.all_hosts()                      # get all hosts that were scanned
#nm[sourceAddress].hostname()          # get one hostname for host 127.0.0.1, usualy the user record
nm[sourceAddress].hostnames()         # get list of hostnames for host 127.0.0.1 as a list of dict [{'name':'hostname1', 'type':'PTR'}, {'name':'hostname2', 'type':'user'}]
nm[sourceAddress].state()             # get state of host 127.0.0.1 (up|down|unknown|skipped) 
nm[sourceAddress].all_protocols()     # get all scanned protocols ['tcp', 'udp'] in (ip|tcp|udp|sctp)
if ('tcp' in nm[sourceAddress]):
    list(nm[sourceAddress]['tcp'].keys()) # get all ports for tcp protocol

nm[sourceAddress].all_tcp()           # get all ports for tcp protocol (sorted version)
nm[sourceAddress].all_udp()           # get all ports for udp protocol (sorted version)
nm[sourceAddress].all_ip()            # get all ports for ip protocol (sorted version)
nm[sourceAddress].all_sctp()          # get all ports for sctp protocol (sorted version)
if nm[sourceAddress].has_tcp(22):     # is there any information for port 22/tcp on host 127.0.0.1
    nm[sourceAddress]['tcp'][22]          # get infos about port 22 in tcp on host 127.0.0.1
    nm[sourceAddress].tcp(22)             # get infos about port 22 in tcp on host 127.0.0.1
    nm[sourceAddress]['tcp'][22]['state'] # get state of port 22/tcp on host 127.0.0.1 (open


# a more usefull example :
for host in nm.all_hosts():
    print('----------------------------------------------------')
    print('Host : {0} ({1})'.format(host, nm[host].hostname()))
    print('State : {0}'.format(nm[host].state()))
    #print('State : {0}'.format(nm[host].os()))

    for proto in nm[host].all_protocols():
        print('----------')
        print('Protocol : {0}'.format(proto))

        lport = list(nm[host][proto].keys())
        lport.sort()
        for port in lport:
            print('port : {0}\tstate : {1}'.format(port, nm[host][proto][port]))


print('----------------------------------------------------')
# print result as CSV
print(nm.csv())



print('----------------------------------------------------')
# If you want to do a pingsweep on network 192.168.1.0/24:
nm.scan(hosts=sourceAddress/24, arguments='-n -sP -PE -PA21,23,80,3389')
hosts_list = [(x, nm[x]['status']['state']) for x in nm.all_hosts()]
for host, status in hosts_list:
    print('{0}:{1}'.format(host, status))





print('----------------------------------------------------')
# Asynchronous usage of PortScannerAsync


nma = nmap.PortScannerAsync()

def callback_result(host, scan_result):
    print('------------------')
    print(host, scan_result)

nma.scan(hosts='192.168.0.0/30', arguments='-sP', callback=callback_result)

while nma.still_scanning():
    print("Waiting ...")
    nma.wait(2)   # you can do whatever you want but I choose to wait after the end of the scan

#if (os.getuid() == 0):
print('----------------------------------------------------')
# Os detection (need root privileges)
nm.scan(sourceAddress, arguments="-O")
if 'osclass' in nm[sourceAddress]:
    for osclass in nm[sourceAddress]['osclass']:
        print('OsClass.type : {0}'.format(osclass['type']))
        print('OsClass.vendor : {0}'.format(osclass['vendor']))
        print('OsClass.osfamily : {0}'.format(osclass['osfamily']))
        print('OsClass.osgen : {0}'.format(osclass['osgen']))
        print('OsClass.accuracy : {0}'.format(osclass['accuracy']))
        print('')

if 'osmatch' in nm[sourceAddress]:
    for osmatch in nm[sourceAddress]['osmatch']:
        print('OsMatch.name : {0}'.format(osclass['name']))
        print('OsMatch.accuracy : {0}'.format(osclass['accuracy']))
        print('OsMatch.line : {0}'.format(osclass['line']))
        print('')

if 'fingerprint' in nm[sourceAddress]:
    print('Fingerprint : {0}'.format(nm[sourceAddress]['fingerprint']))


# Vendor list for MAC address
nm.scan(sourceAddress/24, arguments='-O')
for h in nm.all_hosts():
    if 'mac' in nm[h]['addresses']:
        print(nm[h]['addresses'], nm[h]['vendor'])

        
print('----------------------------------------------------')
# Read output captured to a file
# Example : nmap -oX - -p 22-443 -sV 127.0.0.1 > nmap_output.xml


print(nm.csv())




print('----------------------------------------------------')
# Progressive scan with generator

nm = nmap.PortScannerYield()
for progressive_result in nm.scan(sourceAddress/24, '22-25'):
    print(progressive_result)